In [271]:
import numpy

class Gate:
    def __init__(self, x=None, y=None, x_callback=None, y_callback=None):
        self.dx = None
        self.dy = None
        self.dz = None
        self.z = None
        self.final = None
        self.x_callback = x_callback
        self.y_callback = y_callback
        self.f_callback = None
        if self.x_callback:
            self.x_callback.compute_forward()
            self.x = self.x_callback.z
            self.x_callback.f_callback = self.forward
        else:
            self.x = x
        if self.y_callback:
            self.y_callback.compute_forward()
            self.y = self.y_callback.z
            self.y_callback.f_callback = self.forward
        else:
            self.y = y
        self.compute_forward()
        
    def forward(self):
        self.refresh_inputs()
        self.compute_forward()
        self.update()
        if self.f_callback:
            self.gradients()
            self.f_callback()
        else:
            return(self.z)
        
    def refresh_inputs(self):
        if self.x_callback:
            self.x = self.x_callback.z
        if self.y_callback:
            self.y = self.y_callback.z
        
    def compute_forward(self):
        pass
        
    def backward(self, dz=1):
        self.dz = dz
        self.compute_backward()
        if self.x_callback:
            self.x_callback.backward(self.dx)
        if self.y_callback:
            self.y_callback.backward(self.dy)
        if self.x_callback == None and self.y_callback == None:
            self.forward()
            
    def compute_backward(self):
        pass
    
    def gradients(self):
        return
        print(self.x, self.y)
        print('dx =', self.dx, 'dy =', self.dy)
        
    def update(self):
        # update only if you there is no x or y callback
        if self.x_callback is  None:
            self.x = self.x - self.dx / 2
        if self.y_callback is None:
            self.y = self.y - self.dy / 2
    
class AddGate(Gate):
    def __init__(self, x, y, x_callback=None, y_callback=None):
        super().__init__(x, y, x_callback, y_callback)
    def compute_forward(self):
        self.z = self.x + self.y
    def compute_backward(self):
        self.dx = 1
        self.dy = 1
        
class MultiplyGate(Gate):
    def __init__(self, x, y, x_callback=None, y_callback=None):
        super().__init__(x, y, x_callback, y_callback)
    def compute_forward(self):
        self.z = self.x * self.y
    def compute_backward(self):
        self.dx = self.y * self.dz
        self.dy = self.x * self.dz
        
class PundirGate(Gate):
    def __init__(self, x, y, x_callback=None, y_callback=None):
        super().__init__(x, y, x_callback, y_callback)
    def compute_forward(self):
        self.z = self.x + self.y
    def compute_backward(self):
        self.dx = self.y + (self.dz - self.z)
        self.dy = self.x + (self.dz - self.z)
        
class WeightGate(Gate):
    def __init__(self, x=None, y=None, x_callback=None, y_callback=None):
        super().__init__(x, y, x_callback, y_callback)
    def compute_forward(self):
        self.z = self.x
    def compute_backward(self):
        self.dx = self.dz

In [273]:
w1 = WeightGate(2)
a1 = AddGate(None, 3, w1, None)
w2 = WeightGate(4)
a2 = AddGate(None, 6, w2, None)
a3 = AddGate(None, None, a1, a2)
a3.z

15

In [274]:
a3.backward(a3.z)
print('a3', a3.z, a3.x, a3.y, a3.dz)
print('a2', a2.z, a2.x, a2.y, a2.dz)
print('w2', w2.z, w2.x, w2.y, w2.dz)
print('a1', a1.z, a1.x, a1.y, a1.dz)
print('w1', w1.z, w1.x, w1.y, w1.dz)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [ ]:
hio;
yu9oi'u
